In [193]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn.metrics as metrics
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble.partial_dependence import plot_partial_dependence
#import sklearn.metrics.confusion_matrix
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score

ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [194]:
data = pd.read_csv('F:/Study/methodpro/ml/churn prediction/mini.csv')

In [195]:
data['occurence']= data['occurence'].astype('datetime64') 

In [196]:
data["occurence"] = pd.to_datetime(data["occurence"])

In [197]:
data['year'] = pd.DatetimeIndex(data['occurence']).year
data['month']= pd.DatetimeIndex(data['occurence']).month

In [201]:
data = data[(data['year'] == 2018) & (data['month'] <=7)]
first_6_month = data[data['month'] <=6]
target_month = data[data['month'] == 7]
new_data = pd.DataFrame(first_6_month.groupby('customer_id')['cost'].mean())
new_data['number_of_transactions'] = first_6_month.groupby('customer_id')['cost'].count()
new_data['total'] = first_6_month.groupby('customer_id')['cost'].sum()

In [202]:
new_data.head()

,cost,number_of_transactions,total
customer_id,,,
1,1713.000000,3,5139.0
37,802.733333,45,36123.0
54,5639.500000,10,56395.0
154,2079.000000,4,8316.0
155,1366.666667,3,4100.0


In [203]:
first_6_month = first_6_month[['customer_id']].drop_duplicates(subset=['customer_id'], keep=False)
target_month = target_month[['customer_id']].drop_duplicates(subset=['customer_id'], keep=False)

In [204]:
target_month['target'] = 'yes'
result = pd.merge(first_6_month, target_month, how='left', on=['customer_id'])
result['target'] = (result.target.isnull()).astype('int64')

In [207]:
total = pd.merge(result, new_data, how='inner', on=['customer_id'])
total.tail()

,customer_id,target,cost,number_of_transactions,total
53095,129584,1,2700.0,1,2700.0
53096,195590,1,8390.0,1,8390.0
53097,552969,1,1799.0,1,1799.0
53098,596854,1,1150.0,1,1150.0
53099,506345,1,2190.0,1,2190.0


In [208]:
total.drop(['customer_id'], axis=1)

,target,cost,number_of_transactions,total
0,1,590.0,1,590.0
1,1,500.0,1,500.0
2,1,1390.0,1,1390.0
3,1,3890.0,1,3890.0
4,1,500.0,1,500.0
5,1,1760.0,1,1760.0
6,1,1800.0,1,1800.0
7,1,2100.0,1,2100.0
8,1,4990.0,1,4990.0
9,1,829.0,1,829.0


In [209]:
features=['cost', 'number_of_transactions', 'total']

In [210]:
X = total[features].values
y = total['target'].values

In [211]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [212]:
np.shape(X_train)

(37170, 3)

In [217]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=5, random_state=17)
tree.fit(X_train, y_train)
tree_pred = tree.predict(X_test)

In [219]:
tree.score(X_train, y_train)

0.9520043045466774

In [222]:
tree.score(X_test, y_test)

0.9512241054613936